In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('hpn20-21data.txt')
winter2 = pd.read_csv ('hpn21-22data.txt')
winter3 = pd.read_csv ('hpn22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
193,HPN,2021-12-08 18:23:00,T,SNPL
194,HPN,2021-12-08 18:40:00,T,SNPL
195,HPN,2021-12-08 18:56:00,T,SNPL
201,HPN,2021-12-08 23:56:00,T,-SN BR
202,HPN,2021-12-09 00:56:00,0.25,-SN BR
...,...,...,...,...
2213,HPN,2022-02-25 07:24:00,0.76,-FZRAPL BR
2214,HPN,2022-02-25 07:56:00,2.29,-FZRAPL BR
2215,HPN,2022-02-25 08:56:00,2.79,-SN BR
2216,HPN,2022-02-25 09:05:00,0.25,-FZRAPL BR


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
hpn = LocationInfo(
    'White Plains', 'New York', 
    latitude=41.0340, 
    longitude=-73.7629, 
    timezone='America/New_York'
)

hpn

LocationInfo(name='White Plains', region='New York', timezone='America/New_York', latitude=41.034, longitude=-73.7629)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(hpn.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(hpn.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(hpn.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:01:25  2020-12-01 16:26:59  2020-12-01
1   2020-12-02 07:02:26  2020-12-02 16:26:46  2020-12-02
2   2020-12-03 07:03:26  2020-12-03 16:26:35  2020-12-03
3   2020-12-04 07:04:24  2020-12-04 16:26:26  2020-12-04
4   2020-12-05 07:05:22  2020-12-05 16:26:19  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:38  2021-02-24 17:40:19  2021-02-24
86  2021-02-25 06:35:09  2021-02-25 17:41:30  2021-02-25
87  2021-02-26 06:33:39  2021-02-26 17:42:40  2021-02-26
88  2021-02-27 06:32:08  2021-02-27 17:43:49  2021-02-27
89  2021-02-28 06:30:36  2021-02-28 17:44:59  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:01:11  2021-12-01 16:27:02  2021-12-01
1   2021-12-02 07:02:12  2021-12-02 16:26:48  2021-12-02
2   2021-12-03 07:03:12  2021-12-03 16:26:37  2021-12-03
3   2021-12-04 07:04:10  2021-12-04 16:26:27  2021-12-04
4   2021-12-05 07:05:08  2021-12-05 16:26:20  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:37:00  2022-02-24 17:40:02  2022-02-24
86  2022-02-25 06:35:30  2022-02-25 17:41:12  2022-02-25
87  2022-02-26 06:34:00  2022-02-26 17:42:23  2022-02-26
88  2022-02-27 06:32:30  2022-02-27 17:43:32  2022-02-27
89  2022-02-28 06:30:58  2022-02-28 17:44:42  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:00:56  2022-12-01 16:27:06  2022-12-01
1   2022-12-02 07:01:57  2022-12-02 16:26:51  2022-12-02
2   2022-12-03 07:02:57  2022-12-03 16:26:39  2022-12-03
3   2022-12-04 07:03:56  2022-12-04 16:26:29  2022-12-04
4   2022-12-05 07:04:54  2022-12-05 16:26:22  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:37:21  2023-02-24 17:39:45  2023-02-24
86  2023-02-25 06:35:52  2023-02-25 17:40:55  2023-02-25
87  2023-02-26 06:34:22  2023-02-26 17:42:06  2023-02-26
88  2023-02-27 06:32:52  2023-02-27 17:43:16  2023-02-27
89  2023-02-28 06:31:20  2023-02-28 17:44:25  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2020-12-08 09:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14
1,HPN,2020-12-08 10:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14
2,HPN,2020-12-09 18:56:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:08:57,2020-12-09 16:26:17
3,HPN,2020-12-09 19:56:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:08:57,2020-12-09 16:26:17
4,HPN,2020-12-14 15:56:00,0.5100,PL BR,2020-12-14,2020-12-14 07:12:53,2020-12-14 16:27:05
...,...,...,...,...,...,...,...
153,HPN,2021-02-22 18:56:00,2.7900,SN FG,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
154,HPN,2021-02-23 16:56:00,0.0001,-SNPL BR,2021-02-23,2021-02-23 06:38:07,2021-02-23 17:39:08
155,HPN,2021-02-23 17:56:00,0.7600,-SNRA BR,2021-02-23,2021-02-23 06:38:07,2021-02-23 17:39:08
156,HPN,2021-02-27 07:56:00,0.0001,-SN,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2021-12-08 18:56:00,0.0001,SNPL,2021-12-08,2021-12-08 07:07:53,2021-12-08 16:26:13
1,HPN,2021-12-08 23:56:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:07:53,2021-12-08 16:26:13
2,HPN,2021-12-09 00:56:00,0.2500,-SN BR,2021-12-09,2021-12-09 07:08:45,2021-12-09 16:26:15
3,HPN,2021-12-09 01:56:00,0.0001,-SN BR,2021-12-09,2021-12-09 07:08:45,2021-12-09 16:26:15
4,HPN,2021-12-23 10:56:00,0.0001,-SN,2021-12-23,2021-12-23 07:17:56,2021-12-23 16:30:45
...,...,...,...,...,...,...,...
114,HPN,2022-02-13 16:56:00,0.0001,-SN BR,2022-02-13,2022-02-13 06:52:15,2022-02-13 17:26:48
115,HPN,2022-02-19 17:56:00,0.0001,-SN,2022-02-19,2022-02-19 06:44:12,2022-02-19 17:34:05
116,HPN,2022-02-25 06:56:00,0.2500,-FZRAPL,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12
117,HPN,2022-02-25 07:56:00,2.2900,-FZRAPL BR,2022-02-25,2022-02-25 06:35:30,2022-02-25 17:41:12


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2022-12-11 08:56:00,0.0001,-SN,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
1,HPN,2022-12-11 09:56:00,0.0001,-SN,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
2,HPN,2022-12-11 13:56:00,0.0001,-SN,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
3,HPN,2022-12-11 15:56:00,0.0001,-SN HZ,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
4,HPN,2022-12-11 18:56:00,0.0001,-SN,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
5,HPN,2022-12-11 19:56:00,0.0001,-SN,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
6,HPN,2022-12-11 20:56:00,0.0001,-SN HZ,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
7,HPN,2022-12-11 21:56:00,0.0001,-SN HZ,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
8,HPN,2022-12-11 22:56:00,0.5100,-SN BR,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25
9,HPN,2022-12-11 23:56:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:10:13,2022-12-11 16:26:25


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HPN,2020-12-08 09:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14
1,HPN,2020-12-08 10:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14
4,HPN,2020-12-14 15:56:00,0.5100,PL BR,2020-12-14,2020-12-14 07:12:53,2020-12-14 16:27:05
19,HPN,2020-12-17 07:56:00,1.0200,-SNPL BR,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02
20,HPN,2020-12-17 08:56:00,0.2500,-SNPL BR,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02
...,...,...,...,...,...,...,...
144,HPN,2021-02-19 15:56:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:43:51,2021-02-19 17:34:23
151,HPN,2021-02-22 16:56:00,0.0001,-SNRA,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
154,HPN,2021-02-23 16:56:00,0.0001,-SNPL BR,2021-02-23,2021-02-23 06:38:07,2021-02-23 17:39:08
156,HPN,2021-02-27 07:56:00,0.0001,-SN,2021-02-27,2021-02-27 06:32:08,2021-02-27 17:43:49


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,HPN,2020-12-08 09:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14,2020-12-08 08:08:05,2020-12-08 15:26:14
1,HPN,2020-12-08 10:56:00,0.0001,-SN,2020-12-08,2020-12-08 07:08:05,2020-12-08 16:26:14,2020-12-08 08:08:05,2020-12-08 15:26:14
20,HPN,2020-12-17 08:56:00,0.2500,-SNPL BR,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
21,HPN,2020-12-17 09:56:00,0.2500,SN FZFG,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
22,HPN,2020-12-17 10:56:00,0.0001,SN FZFG,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
23,HPN,2020-12-17 11:56:00,0.2500,SN FZFG,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
24,HPN,2020-12-17 13:56:00,0.0000,BLSN BR,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
25,HPN,2020-12-17 14:56:00,0.0000,BLSN BR,2020-12-17,2020-12-17 07:14:53,2020-12-17 16:28:02,2020-12-17 08:14:53,2020-12-17 15:28:02
40,HPN,2021-01-20 10:56:00,0.0001,-SN,2021-01-20,2021-01-20 07:15:22,2021-01-20 16:57:24,2021-01-20 08:15:22,2021-01-20 15:57:24
41,HPN,2021-01-20 11:56:00,0.0001,-SN,2021-01-20,2021-01-20 07:15:22,2021-01-20 16:57:24,2021-01-20 08:15:22,2021-01-20 15:57:24


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.14524444444444443
2021-2022: 0.3060895833333333
2022-2023: 0.017942857142857142


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,HPN,2020-12-09 18:56:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:08:57,2020-12-09 16:26:17
3,HPN,2020-12-09 19:56:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:08:57,2020-12-09 16:26:17
5,HPN,2020-12-14 16:56:00,0.2500,PL BR,2020-12-14,2020-12-14 07:12:53,2020-12-14 16:27:05
6,HPN,2020-12-14 17:56:00,0.2500,PL BR,2020-12-14,2020-12-14 07:12:53,2020-12-14 16:27:05
7,HPN,2020-12-14 18:56:00,0.5100,-SN BR,2020-12-14,2020-12-14 07:12:53,2020-12-14 16:27:05
...,...,...,...,...,...,...,...
149,HPN,2021-02-19 22:56:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:43:51,2021-02-19 17:34:23
150,HPN,2021-02-19 23:56:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:43:51,2021-02-19 17:34:23
152,HPN,2021-02-22 17:56:00,1.7800,SN FG,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57
153,HPN,2021-02-22 18:56:00,2.7900,SN FG,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:37:57


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_10992\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.27261794871794875
2021-2022: 5.789473684210527e-05
2022-2023: 0.16872
